In [1]:
import sys
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

Using TensorFlow backend.


In [2]:
!wget https://github.com/raviteja8484/ML/blob/master/cities.txt

--2018-06-24 01:05:51--  https://github.com/raviteja8484/ML/blob/master/cities.txt
Resolving github.com (github.com)... 192.30.253.113, 192.30.253.112
Connecting to github.com (github.com)|192.30.253.113|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘cities.txt’

cities.txt              [ <=>                ] 232.69K  --.-KB/s    in 0.05s   

2018-06-24 01:05:51 (4.47 MB/s) - ‘cities.txt’ saved [238276]



In [3]:
ls

cities.txt  datalab/


In [0]:
filename = "cities.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()

In [0]:
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [7]:
n_chars = len(raw_text)
n_vocab = len(chars)
print ("Total Characters: ", n_chars)
print ("Total Vocab: ", n_vocab)

Total Characters:  238170
Total Vocab:  84


In [0]:
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = raw_text[i:i + seq_length]
	seq_out = raw_text[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)

In [10]:
print ("Total Patterns: ", n_patterns)

Total Patterns:  238070


In [0]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [0]:
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))

In [0]:
filepath="weights-tex.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [0]:
import keras

In [0]:
optimizer = keras.optimizers.Adam(lr=0.001)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [0]:
model.fit(X, y, epochs=10, batch_size=128, callbacks=callbacks_list)

Epoch 1/10
238070/238070 [==============================] - 1199s 5ms/step - loss: 2.9154



Epoch 00001: loss improved from inf to 2.91538, saving model to weights-tex.hdf5
Epoch 2/10
 76544/238070 [========>.....................] - ETA: 13:28 - loss: 2.2384

238070/238070 [==============================] - 1190s 5ms/step - loss: 2.1178

Epoch 00002: loss improved from 2.91538 to 2.11778, saving model to weights-tex.hdf5
Epoch 3/10
  3712/238070 [..............................] - ETA: 19:27 - loss: 1.9450

238070/238070 [==============================] - 1193s 5ms/step - loss: 2.1358



Epoch 00003: loss did not improve from 2.11778
Epoch 4/10
 96128/238070 [===========>..................] - ETA: 11:53 - loss: 2.2871

238070/238070 [==============================] - 1198s 5ms/step - loss: 2.0684

Epoch 00004: loss improved from 2.11778 to 2.06842, saving model to weights-tex.hdf5
Epoch 5/10
  9472/238070 [>.............................] - ETA: 19:19 - loss: 1.8610

238070/238070 [==============================] - 1202s 5ms/step - loss: 1.8304



Epoch 00005: loss improved from 2.06842 to 1.83037, saving model to weights-tex.hdf5
Epoch 6/10
 74752/238070 [========>.....................] - ETA: 13:47 - loss: 1.7985

238070/238070 [==============================] - 1203s 5ms/step - loss: 1.7790

Epoch 00006: loss improved from 1.83037 to 1.77900, saving model to weights-tex.hdf5
Epoch 7/10
  3200/238070 [..............................] - ETA: 19:47 - loss: 1.7580

201344/238070 [========================>.....] - ETA: 3:05 - loss: 1.7461

In [0]:
epoch = 10
epochs = 12

model.fit(X, y, epochs=epochs,initial_epoch=epoch, batch_size=128, callbacks=callbacks_list)
model.save_weights("text.h5")
print("Saved model to disk")

In [0]:
filename = "weights-tex.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [0]:
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print ("Seed:")
print ("\"", ''.join([int_to_char[value] for value in pattern]), "\"")

In [0]:
for i in range(100):
	x = numpy.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = numpy.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print "\nDone."